<div id = "monlogo"><center> <img src="https://automobilist.com/cdn/shop/files/ATM_Partners_F1_f1eeb8dc-dd0d-45cf-a96f-55414ecda12e.png?v=1697534842&width=300", style="height:150px"></center></div>

<hr style="border-width:2px;border-color:#FF0000"><br>
<center> <h1>Formula One</h1> </center> <br>
<center> <h3>Machine Learning Forecasting</h3> </center> <br>
<hr style="border-width:2px;border-color:#FF0000"> 


#### **For each race of the 2021 season, who is the winner ?**

In [3]:
import pandas as pd
from skimpy import skim
import numpy as np

import optuna #  hyperparameter optimization library / Bayesian optimization
from sklearn.preprocessing import LabelEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score

from xgboost import XGBClassifier

In [8]:
# Data import
results = pd.read_csv("data/results.csv", sep=',')
races = pd.read_csv("data/races.csv", sep=',')
circuits = pd.read_csv("data/circuits.csv", sep=',')
constructors = pd.read_csv("data/constructor_standings.csv", sep=',')
pit_stops =  pd.read_csv("data/pit_stops.csv", sep=',')
status = pd.read_csv("data/status.csv", sep=',')
weather =  pd.read_csv("data/weather.csv", sep=',')
lap_times = pd.read_csv("data/lap_times.csv")

In [9]:
# Merging
df1 = pd.merge(results[["driverId","constructorId","raceId",
            "grid","positionOrder","fastestLapSpeed","statusId"]].merge(status, on="statusId"),races[["raceId", "year", "circuitId"]],on="raceId")

df1_1 = df1.merge(weather.drop_duplicates(), on=["year", "raceId"], how="inner")

df2 = pd.merge(
    df1_1,constructors[["raceId", "constructorId", "position", "points"]],
    on=["raceId", "constructorId"])

df3 = df2.merge(circuits[["circuitId", "country"]], on="circuitId")

df3["winner"] = df3["positionOrder"].apply(lambda x: 1 if x == 1 else 0)
df4=df3.merge(pit_stops[["raceId","driverId","duration"]].rename(columns={"duration":"duration_pit"}),on=["raceId","driverId"],how="left")
df4=df4.fillna(0)


df4=df4.drop(["status",'circuitId', 'constructorId'],axis=1).set_index(['raceId', 'driverId'])
df4

grid  positionOrder fastestLapSpeed  statusId  year  \
raceId driverId                                                        
18     1            1            1.0         218.300         1  2008   
       2            5            2.0         217.586         1  2008   
       3            7            3.0         216.719         1  2008   
       4           11            4.0         215.464         1  2008   
       5            3            5.0         218.385         1  2008   
...               ...            ...             ...       ...   ...   
1084   832          3            0.0         226.438       131  2022   
       832          3            0.0         226.438       131  2022   
       849         17            0.0         219.280       140  2022   
       849         17            0.0         219.280       140  2022   
       815          5            0.0         216.371       130  2022   

                 weather_warm  weather_cold  weather_dry  weather_wet  \
raceId driverId                                                         
18     1                    0             0            0            0   
       2                    0             0            0            0   
       3                    0             0            0            0   
       4                    0             0            0            0   
       5                    0             0            0            0   
...                       ...           ...          ...          ...   
1084   832                  0             0            0            0   
       832                  0             0            0            0   
       849                  0             0            0            0   
       849                  0             0            0            0   
       815                  0             0            0            0   

                 weather_cloudy  position  points    country  winner  \
raceId driverId                                                        
18     1                      0         1    14.0  Australia       1   
       2                      0         3     8.0  Australia       0   
       3                      0         2     9.0  Australia       0   
       4                      0         4     5.0  Australia       0   
       5                      0         1    14.0  Australia       0   
...                         ...       ...     ...        ...     ...   
1084   832                    1         2   303.0    Austria       0   
       832                    1         2   303.0    Austria       0   
       849                    1        10     3.0    Austria       0   
       849                    1        10     3.0    Austria       0   
       815                    1         1   359.0    Austria       0   

                duration_pit  
raceId driverId               
18     1                   0  
       2                   0  
       3                   0  
       4                   0  
       5                   0  
...                      ...  
1084   832            21.212  
       832            21.492  
       849            21.722  
       849            21.206  
       815            23.681  

[22916 rows x 15 columns]

In [10]:
df4.dtypes

grid                 int64
positionOrder      float64
fastestLapSpeed     object
statusId             int64
year                 int64
weather_warm         int64
weather_cold         int64
weather_dry          int64
weather_wet          int64
weather_cloudy       int64
position             int64
points             float64
country             object
winner               int64
duration_pit        object
dtype: object

Variable country

In [11]:
label_encoder = LabelEncoder()
df4.loc[:, 'country'] = label_encoder.fit_transform(df4['country'])
df4['country'] = df4['country'].astype(int)


Variable duration pit

In [12]:
df4['duration_pit']=df4['duration_pit'].astype(str)
# df4['duration_pit'].to_csv('duration_pit.csv')
# there are different values (like 19:37.545) among the floats, they must be transformed into floats using an if condition:
def convert_time_to_float(time_str):
    if ':' in time_str:
        minutes, seconds = time_str.split(':')
        minutes = int(minutes)
        seconds = float(seconds)
        decimal_time = minutes+seconds/60
        return decimal_time
    else:
        return float(time_str)
# filter application
df4['duration_pit'] = df4['duration_pit'].apply(convert_time_to_float)
df4['duration_pit']=df4['duration_pit'].astype(float)

Variable fastest Lap Speed

In [13]:
df4['fastestLapSpeed']=df4['fastestLapSpeed'].astype(str)
# df4['fastestLapSpeed'].to_csv('fastestLapSpeed.csv')
# here the problem is the \N in the data
# they must be replaced by Nan
df4['fastestLapSpeed']=df4['fastestLapSpeed'].replace('\\N',np.nan)
df4['fastestLapSpeed']=df4['fastestLapSpeed'].astype(float)


In [16]:
df_train = df4[df4['year']<=2020]
df_test = df4[df4['year']==2021]

X_train = df_train[['grid',"position",'points','country','fastestLapSpeed','duration_pit',
       'weather_warm','weather_cold', 'weather_dry', 'weather_wet', 'weather_cloudy']]
X_test = df_test[['grid',"position",'points','country','fastestLapSpeed','duration_pit',
       'weather_warm','weather_cold', 'weather_dry', 'weather_wet', 'weather_cloudy']]

y_train = df_train[['winner']]
# y_pred contains no information, this is what needs to be predicted
y_pred = df_test[['winner']]

In [127]:
def objective(trial):
    # space of hyperparameters to test
    param = {
        'n_estimators':trial.suggest_int('n_estimators',50, 300),
        'max_depth':trial.suggest_int('max_depth',2, 50), 
        'learning_rate':trial.suggest_float('learning_rate',0.01,0.3),
        'min_child_weight':trial.suggest_int('min_child_weight',1,10), # number of observations needed to continue a node
        'gamma': trial.suggest_float('gamma',0.0,5.0),
        'subsample': trial.suggest_float('subsample',0.5,1.0)
    }
    
    xgb_model = XGBClassifier(**param, random_state=42)
    
    # cross-validation: f1 average score with 3 folds
    f1_mean = cross_val_score(xgb_model,X_train,y_train,n_jobs=-1,cv=3,scoring='f1').mean() # use of all available cores

    return f1_mean

In [128]:
# study optuna
study = optuna.create_study(direction='maximize') # goal : to maximize the score
study.optimize(objective, n_trials=100)

# best parameters
best_params = study.best_params
print('Meilleurs hyperparamètres:', best_params)

# training the model with the best parameters
best_xgb_model = XGBClassifier(**best_params, random_state=42)
best_xgb_model.fit(X_train, y_train)

[I 2024-03-08 16:01:53,818] A new study created in memory with name: no-name-4c1c8471-51bd-47e7-970a-9424f206894c


[I 2024-03-08 16:01:54,369] Trial 0 finished with value: 0.3149993246115277 and parameters: {'n_estimators': 154, 'max_depth': 11, 'learning_rate': 0.22428224705005279, 'min_child_weight': 8, 'gamma': 2.3845089574413665, 'subsample': 0.9145931505799497}. Best is trial 0 with value: 0.3149993246115277.
[I 2024-03-08 16:01:54,750] Trial 1 finished with value: 0.3096771902243339 and parameters: {'n_estimators': 61, 'max_depth': 20, 'learning_rate': 0.08484473999723709, 'min_child_weight': 8, 'gamma': 1.305966409543728, 'subsample': 0.6999551955065453}. Best is trial 0 with value: 0.3149993246115277.
[I 2024-03-08 16:01:55,540] Trial 2 finished with value: 0.31464194594139777 and parameters: {'n_estimators': 222, 'max_depth': 15, 'learning_rate': 0.07327873057695379, 'min_child_weight': 10, 'gamma': 0.2786472500455428, 'subsample': 0.7973674768450927}. Best is trial 0 with value: 0.3149993246115277.
[I 2024-03-08 16:01:56,018] Trial 3 finished with value: 0.3370321974951167 and parameters:

Meilleurs hyperparamètres: {'n_estimators': 273, 'max_depth': 2, 'learning_rate': 0.283230429159712, 'min_child_weight': 6, 'gamma': 4.685903386699535, 'subsample': 0.9585762547505405}


XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=4.685903386699535, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=0.283230429159712,
              max_bin=None, max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=2, max_leaves=None,
              min_child_weight=6, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=273, n_jobs=None,
              num_parallel_tree=None, random_state=42, ...)

- Best trial 51 : 0.409. (Best F1 Score)
- Best hyperparameters : {'n_estimators': 273, 'max_depth': 2, 'learning_rate': 0.283230429159712, 'min_child_weight': 6, 'gamma': 4.685903386699535, 'subsample': 0.9585762547505405}

In [130]:
predictions = best_xgb_model.predict(X_test)
y_pred.loc[:,'winner'] = predictions

y_pred.loc[:,'positionOrder'] = 0
# csv export
y_pred.to_csv('y_pred_logreg.csv')